#### Chain Of Thoughts With RAG
What is Chain-of-Thought (CoT) in RAG?

CoT reasoning breaks down a complex question into intermediate steps, and allows retrieval + reflection at each step before answering.

User Query
   ↓
- Step 1: Decompose question → sub-steps (Reason)
- Step 2: Retrieve docs per step (Act)
- Step 3: Combine context (Observe)
- Step 4: Final answer generation (Reflect)

In [4]:
import os
from typing import List
from pydantic import BaseModel
from langchain.schema import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph, END

In [5]:
# -------------------------------
# 1. Prepare Vectorstore
# -------------------------------
docs = TextLoader("data/research_notes.txt",encoding="utf-8").load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever()

In [6]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = init_chat_model(model="openai:gpt-4o-mini")

In [8]:
# -------------------------------
# 2. LangGraph State Definition
# -------------------------------
class RAGCotState(BaseModel):
    question: str
    sub_steps: List[str] = []
    retrieved_docs: List[Document] = []
    answer: str = ""

In [9]:
# -------------------------------
# 3. Nodes
# -------------------------------

# a. Plan sub-questions
def plan_steps(state:RAGCotState)->RAGCotState:
    prompt = f"Break the question into 2-3 reasoning steps: \n\n {state.question}"
    result = llm.invoke(prompt).content
    subqs=[line.strip("- ") for line in result.split("\n") if line.strip()]

    return state.model_copy(update={"sub_steps":subqs})

In [10]:
#b. Retrive for each step
def retrieve_per_step(state:RAGCotState)->RAGCotState:
    all_docs = []
    for sub in state.sub_steps:
        docs = retriever.invoke(sub)
        all_docs.extend(docs)
    return state.model_copy(update={"retrieved_docs":all_docs})

In [11]:
# c. Generate Final Answer
def generate_answer(state: RAGCotState) -> RAGCotState:
    
    context = "\n\n".join([doc.page_content for doc in state.retrieved_docs])
    prompt = f"""
You are answering a complex question using reasoning and retrieved documents.

Question: {state.question}

Relevant Information:
{context}

Now synthesize a well-reasoned final answer.
"""
    result = llm.invoke(prompt).content.strip()
    return state.model_copy(update={"answer": result})

In [12]:
# -------------------------------
# 4. LangGraph Graph
# -------------------------------
builder = StateGraph(RAGCotState)
builder.add_node("planner", plan_steps)
builder.add_node("retriever", retrieve_per_step)
builder.add_node("responder", generate_answer)

builder.set_entry_point("planner")
builder.add_edge("planner", "retriever")
builder.add_edge("retriever", "responder")
builder.add_edge("responder", END)

graph = builder.compile()

In [15]:
# -------------------------------
# 5. Run CoT RAG Agent
# -------------------------------
if __name__ == "__main__":
    query = "what are the additional eperiments in Transformer eveluation?"
    state = RAGCotState(question=query)
    final = graph.invoke(state)

    print("\n🪜 Reasoning Steps:", final["sub_steps"])
    print("\n✅ Final Answer:\n", final["answer"])


🪜 Reasoning Steps: ['To break down the question, "What are the additional experiments in Transformer evaluation?" into reasoning steps, we can approach it as follows:', '1. **Identify the Purpose of Transformer Evaluation**: Understand what aspects of Transformer models need to be evaluated. This includes examining their performance in tasks like natural language processing (NLP), image processing, or other specific applications.', '2. **Explore Existing Evaluation Metrics and Methods**: Look into the common metrics and benchmarks used to evaluate Transformers, such as accuracy, F1 scores, BLEU scores for translation tasks, and others. Identify gaps or areas where additional experimentation could provide deeper insights.', '3. **Propose Possible Additional Experiments**: Consider new or underexplored aspects that could be evaluated, such as robustness to adversarial examples, efficiency in terms of speed and resource usage, cross-linguistic capabilities, or performance in low-data sce